In [16]:
import numpy as np
import pandas as pd
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


ab=pd.read_csv("en_train.csv")
df=pd.DataFrame(ab)
df=df.sample(n=20000,random_state=40)

class_transform = {"PLAIN":0, "PUNCT":1, "DATE":2, "LETTERS":3, "CARDINAL":4, "VERBATIM":5,
       "DECIMAL":6, "MEASURE":7, "MONEY":8, "ORDINAL":9, "TIME":10, "ELECTRONIC":11,
       "DIGIT":12, "FRACTION":13, "TELEPHONE":14, "ADDRESS":15}
df['class']=df['class'].apply(class_transform.get).astype(int)
df['change'] = 1
df.loc[df.before == df.after, 'change'] = 0


df1=df[['class','before','after','change']]

df_list = [0 for x in range(0,16)]

for x in range(0,16):
    df_list[x] = df1[df1['class']==x]



warnings.filterwarnings("ignore")
dict2 = [{} for x in range(0,16)]

for x in range(0,16):
    arr,ind=pd.factorize(df_list[x].before)
    df_list[x]['before_id'] = arr
    arr,ind=pd.factorize(df_list[x].after)
    df_list[x]['after_id'] = arr
    for y in range(len(ind)):
        dict2[x][y] = ind[y]

df2=pd.concat([df_list[0],df_list[1],df_list[2],df_list[3],df_list[4],df_list[5],df_list[6],df_list[7],df_list[8],df_list[9],df_list[10],df_list[11],df_list[12],df_list[13],df_list[14],df_list[15]])

X_train, X_test, y_train, y_test = train_test_split(df2[['before','class','before_id']], df2[['after','change','after_id']], test_size = 0.2)


In [17]:
x_train1 = [0 for x in range(0,16)]
x_test1 = [0 for x in range(0,16)]
y_train1 = [0 for x in range(0,16)]
y_test1 = [0 for x in range(0,16)]
df_list1 = [0 for x in range(0,16)]
list_inputs = [[] for x in range(0,16)]
list_outputs = [[] for x in range(0,16)]
model = [0 for x in range(0,16)]
model1 = [0 for x in range(0,16)]
predict = [0 for x in range(0,16)]

for x in range(0,16):
    print(x)
    x_train1[x] = X_train[X_train['class']==x]
    y_train1[x] = y_train[X_train['class']==x]
    
    if x_train1[x].shape[0]==0:
        continue
    
    model[x] = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10,10,10,10), activation='tanh',random_state=60)
    model[x].fit(x_train1[x][['before_id']],y_train1[x][['change']])
    for y in range(0,16):
        x_train1[x]['a'+str(y)]=0
        list_inputs[x].append('a'+str(y))

    for y in range(0,16):
        y_train1[x]['b'+str(y)]=0
        list_outputs[x].append('b'+str(y))
    
    if x_train1[x].shape[0]==0:
        continue
    
    x_train1[x]['change']=model[x].predict(x_train1[x][['before_id']])
    x_train1[x]=x_train1[x][x_train1[x]['change']==1]
    
    if x_train1[x].shape[0]==0:
        continue
    
    y_train1[x]=y_train1[x].loc[x_train1[x].index.tolist()]
    
    
    for y in x_train1[x].index.tolist():
        temp = x_train1[x].loc[y,'before_id']
        list1=[]
        while temp!=0:
            list1.append(temp%2)
            temp = temp//2

        for z in range(0,len(list1)):
            x_train1[x].loc[y, 'a'+str(z)] = list1[z]

        temp = y_train1[x].loc[y,'after_id']
        list1=[]
        while temp!=0:
            list1.append(temp%2)
            temp = temp//2

        for z in range(0,len(list1)):
            y_train1[x].loc[y, 'b'+str(z)] = list1[z]
    
    model1[x] = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(50,50,50,50), activation='tanh',random_state=90)
    model1[x].fit(x_train1[x][list_inputs[x]],y_train1[x][list_outputs[x]])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [18]:
def testing(X_test,y_test):
    for x in range(0,16):
        print(x)
        x_test1[x] = X_test[X_test['class']==x]
        y_test1[x] = y_test[X_test['class']==x]

        if x_test1[x].shape[0]==0:
            continue

        x_test1[x]['change_predict'] = model[x].predict(x_test1[x][['before_id']])

        if x_test1[x][x_test1[x]['change_predict']==1].shape[0]==0:
            continue

        l=[]
        l1=[]
        for y in range(0,16):
            x_test1[x]['a'+str(y)]=0
            x_test1[x]['b'+str(y)]=0
            l.append('a'+str(y))
            l1.append('b'+str(y))
        x_test1[x]['predict']=0

        for y in x_test1[x][x_test1[x]['change_predict']==1].index.tolist():
            temp = x_test1[x].loc[y,'before_id']
            list1=[]
            while temp!=0:
                list1.append(temp%2)
                temp = temp//2

            for z in range(0,len(list1)):
                x_test1[x].loc[y, 'a'+str(z)] = list1[z]


        x_test1[x][x_test1[x]['change_predict']==1][l1] = model1[x].predict(x_test1[x][x_test1[x]['change_predict']==1][l]).astype(int)

        for y in x_test1[x][x_test1[x]['change_predict']==1].index.tolist():
            val = 0
            p = 1
            for z in range(0,16):
                val = val+x_test1[x].loc[y,l1[z]]*p
                p = p*2;
            x_test1[x].loc[y,'predict'] = val
        x_test1[x]['predict'] = x_test1[x]['predict'].astype(int)

    for x in range(0,16):
        if x_test1[x].shape[0]==0:
            continue

        x_test1[x]['after_predict'] = x_test1[x]['before']

        for y in x_test1[x][x_test1[x]['change_predict']==1].index.tolist():
            if x_test1[x].loc[y,'predict'] in dict2[x].keys():
                x_test1[x].loc[y,'after_predict'] = dict2[x][x_test1[x].loc[y,'predict']]

    input1 =pd.concat([x_test1[0],x_test1[1],x_test1[2],x_test1[3],x_test1[4],x_test1[5],x_test1[6],x_test1[7],x_test1[8],x_test1[9],x_test1[10],x_test1[11],x_test1[12],x_test1[13],x_test1[14],x_test1[15]])
    output1 =pd.concat([y_test1[0],y_test1[1],y_test1[2],y_test1[3],y_test1[4],y_test1[5],y_test1[6],y_test1[7],y_test1[8],y_test1[9],y_test1[10],y_test1[11],y_test1[12],y_test1[13],y_test1[14],y_test1[15]])

    input1 = input1[['before','after_predict']]
    output1 = output1['after']

    print("Accuracy : "+str(input1[input1['after_predict']==output1].shape[0]*100/input1.shape[0]))

In [19]:
print("Training : ")
print("Number of instances : "+str(X_train.shape[0]))
testing(X_train,y_train)
print("Testing : ")
print("Number of instances : "+str(X_test.shape[0]))
testing(X_test,y_test)

Training : 
Number of instances : 16000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Accuracy : 93.46875
Testing : 
Number of instances : 4000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Accuracy : 93.35
